### **0. Set-up**

In [1]:
# Import libraries and utils
%run '../../utils.ipynb'

In [2]:
#  Get api key
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set client
client = OpenAI()

In [3]:
# Load dataframe
cleaned_nl_simlex = pd.read_csv("../../../data/dataset/cleaned-nl-simlex-999.csv")

# Select subset
# cleaned_nl_simlex = cleaned_nl_simlex.head(150)

# Convert to tuple
tuples_list = list(zip(cleaned_nl_simlex['word1'], cleaned_nl_simlex['word2']))

In [4]:
# Show results
cleaned_nl_simlex

,word1,word2,SimLex999,POS
0,oud,nieuw,1.94,A
1,slim,intelligent,8.19,A
2,hard,moeilijk,4.46,A
3,gelukkig,vrolijk,6.49,A
4,hard,stoer,5.69,A
...,...,...,...,...
992,samenvoegen,verwerven,3.89,V
993,sturen,bijwonen,1.85,V
994,verzamelen,bijwonen,1.06,V
995,opnemen,intrekken,2.29,V


### **1. Define and Evaluate Parameters**

In [5]:
# Define the prompt
prompt = ("Twee woorden zijn synoniemen als ze zeer vergelijkbare betekenissen hebben. Synoniemen vertegenwoordigen hetzelfde type of dezelfde categorie van dingen. "
          "Hier zijn enkele voorbeelden van synoniemparen zijn: kop/mok, buurt/wijk, jaloezie/afgunst. "
          "In de praktijk kunnen woordparen die niet exact synoniem zijn, toch zeer vergelijkbaar zijn. Hier zijn enkele bijna synoniemen: alligator/krokodil, liefde/genegenheid, kikker/pad. "
          "In tegenstelling hiermee zijn de volgende woordparen wel gerelateerd, maar niet erg vergelijkbaar. De woorden vertegenwoordigen totaal verschillende soorten dingen: auto/band, auto/snelweg, auto/ongeluk. "
          "Beoordeel de semantische gelijkenis van elk woordpaar op een schaal van 0 tot 10, waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. "
          "Onthoud dat het hier gaat om gelijkenis en niet om gerelateerdheid. "
          "Er is geen juist antwoord op deze vragen. Het is volkomen redelijk om je intuïtie of onderbuikgevoel te gebruiken als Nederlandstalige, vooral wanneer je wordt gevraagd om woordparen te beoordelen die je niet vergelijkbaar vindt. "
          "Het antwoord moet strikt voldoen aan de structuur: [('woord1', 'woord2', <score>), ('woord3', 'woord4', <score>), ...]. Gebruik twee decimalen. Geef geen extra uitleg of context.")

In [6]:
# Define model
model = "gpt-3.5-turbo-0125"

# Set sample size
sample_size = 15
# sample_size = 5

# Delay between individual API calls
delay = 10.0

# Define number of sublists
n_sublists = 60
# n_sublists = 15

In [7]:
# Split the list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count the number of lists
print(len(chunks))

60


In [8]:
# Print the prompts for each chunk
print_prompts(chunks, prompt)

Twee woorden zijn synoniemen als ze zeer vergelijkbare betekenissen hebben. Synoniemen vertegenwoordigen hetzelfde type of dezelfde categorie van dingen. Hier zijn enkele voorbeelden van synoniemparen zijn: kop/mok, buurt/wijk, jaloezie/afgunst. In de praktijk kunnen woordparen die niet exact synoniem zijn, toch zeer vergelijkbaar zijn. Hier zijn enkele bijna synoniemen: alligator/krokodil, liefde/genegenheid, kikker/pad. In tegenstelling hiermee zijn de volgende woordparen wel gerelateerd, maar niet erg vergelijkbaar. De woorden vertegenwoordigen totaal verschillende soorten dingen: auto/band, auto/snelweg, auto/ongeluk. Beoordeel de semantische gelijkenis van elk woordpaar op een schaal van 0 tot 10, waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. Onthoud dat het hier gaat om gelijkenis en niet om gerelateerdheid. Er is geen juist antwoord op deze vragen. Het is volkomen redelijk om je intuïtie of onderbuikgevoel te gebruiken als Nederla

In [9]:
# Load the encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count the tokens per chunk
token_counts = count_tokens_with_tiktoken(chunks, prompt)
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [544, 546, 553, 547, 539, 541, 540, 535, 529, 520, 528, 534, 531, 534, 526, 534, 529, 532, 529, 534, 542, 534, 535, 530, 530, 541, 528, 526, 538, 542, 542, 546, 531, 553, 544, 535, 533, 509, 528, 524, 530, 524, 550, 527, 530, 518, 541, 540, 533, 546, 538, 535, 549, 546, 550, 547, 543, 536, 542, 552]


In [10]:
# Number of total tokens
sum(token_counts)

32173

### **2. Extract and Process Data**

In [11]:
# FINAL RUNNING

In [12]:
# Process each chunk and get results using the OpenAI API
response = get_responses(chunks, prompt, model, sample_size, delay)

Processing:   0%|          | 0/900 [00:00<?, ?chunk/s]

Processing: 100%|██████████| 900/900 [3:51:25<00:00, 15.43s/chunk]  

Total time taken: 13885.66 seconds


In [13]:
# Define filepath
file_path = '../../../data/response/nl/gpt-3.5-turbo-0125/f7.json'

# Check if the file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


In [14]:
# Extract data with regular expressions into dictionary
data_dict = process_responses(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}
print(higher_lower_samples)

{}


In [15]:
# Process data and print duplicate word pairs
print_duplicate_word_pairs(cleaned_nl_simlex, data_dict)

Empty DataFrame
Columns: [Combined_Columns]
Index: []
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [16]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,oud,nieuw,1.0,0.1,1.0,0.10,1.0,1.00,2.00,0.1,2.00,2.00,0.10,0.1,0.10,0.2,1.00
1,slim,intelligent,1.0,0.9,0.8,0.95,0.9,0.90,8.00,0.9,7.00,8.00,0.90,0.9,0.95,0.9,0.90
2,hard,moeilijk,0.0,0.2,0.2,0.30,0.2,0.20,4.00,0.2,2.00,3.00,0.20,0.2,0.20,0.4,0.30
3,gelukkig,vrolijk,0.5,0.8,0.6,0.80,0.8,0.70,6.00,0.8,6.50,8.00,0.70,0.7,0.80,0.8,0.80
4,hard,stoer,0.2,0.5,0.3,0.50,0.4,0.40,4.00,0.5,4.00,4.00,0.50,0.4,0.60,0.6,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,samenvoegen,verwerven,0.0,1.0,2.0,0.10,0.1,0.05,0.20,1.0,0.05,0.10,0.10,0.2,0.20,1.0,0.30
993,sturen,bijwonen,0.0,1.0,1.0,0.10,0.2,0.10,0.05,1.0,0.05,0.10,0.05,0.1,0.10,1.0,0.05
994,verzamelen,bijwonen,0.0,2.0,2.0,0.20,0.2,0.20,0.10,2.0,0.05,0.10,0.10,0.3,0.20,1.0,0.10
995,opnemen,intrekken,0.0,1.0,1.0,0.10,0.2,0.10,0.20,1.0,0.10,0.05,0.10,0.2,0.10,1.0,0.10


In [17]:
# Count null values
count_null_values = df.isnull().sum()
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                  0
word2                  0
similarity_score_1     0
similarity_score_2     0
similarity_score_3     0
similarity_score_4     0
similarity_score_5     0
similarity_score_6     0
similarity_score_7     0
similarity_score_8     0
similarity_score_9     0
similarity_score_10    0
similarity_score_11    0
similarity_score_12    0
similarity_score_13    0
similarity_score_14    0
similarity_score_15    0
dtype: int64


In [18]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15


In [19]:
# Define file_path
file_path = '../../../data/prompt/nl/gpt-3.5-turbo-0125/f7.csv'

# Check if the file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")

File saved successfully.
